<a href="https://colab.research.google.com/github/dssalazarp/SyS/blob/main/Copia_de_Parcial1_SyS_2024_2_David_Salazar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1: Señales y Sistemas 2024-II

 ## Profesor: Andrés Marino Álvarez Meza, Ph.D.
 ## Estudiante: David Santiago Salazar Pineda

## Departamento de Ingeniería Eléctrica, Electrónica, y Computación
## Universidad Nacional de Colombia - sede Manizales

# Instrucciones

-- Para recibir el crédito total, sus respuestas deben estar justificadas de manera clara, detallada y concreta, mostrando los procedimientos y razonamientos paso a paso.

-- Está permitido el uso de herramientas de inteligencia artificial (IA). Si las utiliza, por favor declare explícitamente cómo fueron empleadas en la resolución de cada pregunta. Incluya los prompts (consultas) y las iteraciones realizadas con las IA durante el desarrollo del parcial.

-- La entrega del parcial debe realizarse antes de las 23:59 del 5 de diciembre de 2024 al correo electrónico amalvarezme@unal.edu.co mediante un enlace de GitHub.

-- Los códigos deben estar debidamente comentados en las celdas correspondientes y explicados en celdas de texto (markdown). Los códigos que no incluyan comentarios ni discusiones no serán considerados en la evaluación final.

# Pregunta 1 (valor 2.5 puntos)

Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital de 5 bits con frecuencia de muestreo de $5kHz$, aplicado a la señal continua $x(t) = 0.3 \cos(1000\pi t-\pi/4) +
0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t-\pi)$?. Realizar la simulación del proceso de digitalización incluyendo al menos 3 ciclos de la señal $x(t)$.

En caso de que la digitalización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada. El convesor debe permitir configurar la cantidad de bits y la frecuencia de muestreo, indicándole al usuario si dicha frecuencia es apropiada o no, y graficar la señal continua, discreta y digital.

Respuesta:



1.   Desarrollo de la funcion dada bajos los parametros establesitos por el ejercicio con la señal continua $x(t) = 0.3 \cos(1000\pi t-\pi/4) +
0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t-\pi)$ y frecuencia de muestreo de $5kHz$. incluyendo representacion grafica de la misma



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# Definir parámetros de la señal
# Amplitud
A1 = 0.3
A2 = 0.6
A3 = 0.1

# Frecuencia angular
w1 = 1000 * np.pi
w2 = 2000 * np.pi
w3 = 11000 * np.pi

# Periodo de las señales
T1 = 2 * np.pi / w1
T2 = 2 * np.pi / w2
T3 = 2 * np.pi / w3


# Frecuencia de las señales
F1 = w1 / (2 * np.pi)
F2 = w2 / (2 * np.pi)
F3 = w3 / (2 * np.pi)

# Definir frecuencia de muestreo = 5kHz
Fs = 5000

# Calcular el MCM de los períodos para hallar el periodo total de la funcion cuasi periodica
TT = np.lcm.reduce([int(np.round(T1 * 10000)), int(np.round(T2 * 10000)), int(np.round(T3 * 10000))]) / 10000
# se multipica y divide por un numero muy grande para evitar numeros fracionarios debido a que la funcion np.lcm.reduce() no los admite https://numpy.org/doc/2.1/reference/generated/numpy.lcm.html biblioteca de tla funcion
Ts= 1/Fs

# Definir el rango de tiempo vector tiempo de 0 hasta 3 periodos
tv = np.arange(0,3*TT , Ts)

# Construir la señal
X = A1 * np.cos(w1 * tv - (np.pi / 4)) + A2 * np.sin(w2 * tv) + A3 * np.cos(w3 * tv - np.pi)

# Graficar la señal continua original
plt.plot(tv, X, 'r', label='Señal continua $x(t)$')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Señal continua $x(t)$')
plt.legend()
plt.grid(True)
plt.show()


# se grafica la funcion orginal para tener un contraste con respuestas futuras

In [ ]:
def my_ceropen(x,ymin = -5, ymax = 5): #en general se pueden definir valores por defecto
  '''
  Codigo base para simular proceso de cero y pendiente
  Se ingresa arreglo de numpy y los valores min  y max despues de cero y pendiente
  '''
  xmax = max(x) # x.max()
  xmin = min(x) # x.min()
  m = (ymax - ymin)/(xmax - xmin)
  c = ymin - m*xmin
  yv = m*x + c
  return yv

In [ ]:
ymin=-3.3
ymax=3.3
xcp= my_ceropen(X,ymin=ymin,ymax=ymax)
plt.plot(tv,xcp,c='r',label='$cero_pendiente$')
plt.legend()
plt.show()

In [ ]:
xcp.shape
tv.shape

In [ ]:
nbits=5
num_est=2**nbits
vq = np.linspace(ymin,ymax,num_est)
vq

In [ ]:
def my_cuantizador(yn, vq) : #yn punto a #cuantizar, vq vector de estados

  Ne = vq.shape[0] #tamaño vector de estados
  dn = cdist(yn.reshape(-1,1),vq.reshape(-1,1))#distancia yn a vector estados, reshape(-1,1) asegura vectores columna para poder utilizar cdist
  ind = np.argmin(dn) #posición distancia min
  return vq[ind]

In [ ]:
#función cuantización punto a punto
import numpy as np
import matplotlib.pyplot as plt


#____________________________________________________________________________
for i in range(xcp.shape[0]):
  if i==0: #primera muestra
    yq = my_cuantizador(xcp[i],vq)
  else:
    yq = np.append(yq,my_cuantizador(xcp[i],vq)) #apilar datos cuantizando si i>0
#_____________________________________________________________________________
#graficar señales

plt.plot(tv,xcp,c='r', label='ycs')#cs continua

plt.stem(tv,xcp, label='ycs_dis')#cs discreta

#estados de cuantización

for v in vq:
  plt.plot(tv,v*np.ones(len(tv)))

# graficar señal Digital (discreta + cuantizada)

plt.plot(tv,yq,c='y',marker='o',label='ydig', linewidth=2)

plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.show()

# NOTA
 se puede observar que no cumple con el criterio de Nyquits debido a que existe una señal la cual supera la frecuencia de muestreo de 5000 hz la cual es de 5500hz

se propone una nueva frecuencia de muestrero la cual sera 4 veces mayor a la frecuencia maxima entre las señales


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# Definir parámetros de la señal
# Amplitud
A1 = 0.3
A2 = 0.6
A3 = 0.1

# Frecuencia angular
w1 = 1000 * np.pi
w2 = 2000 * np.pi
w3 = 11000 * np.pi

# Periodo de las señales
T1 = 2 * np.pi / w1
T2 = 2 * np.pi / w2
T3 = 2 * np.pi / w3


# Frecuencia de las señales
F1 = w1 / (2 * np.pi)
F2 = w2 / (2 * np.pi)
F3 = w3 / (2 * np.pi)

# Definir frecuencia de muestreo
Fn= max([F1,F2,F3]) #busca la frecuencia mas alta en la funcion
Fs = 4*Fn # freecuencia de Nyquist

# Calcular el MCM de los períodos
TT = np.lcm.reduce([int(np.round(T1 * 10000)), int(np.round(T2 * 10000)), int(np.round(T3 * 10000))]) / 10000
Ts= 1/Fs
# Definir el rango de tiempo vector tiempo
tv = np.arange(0,3*TT , Ts)

# Construir la señal
X = A1 * np.cos(w1 * tv - (np.pi / 4)) + A2 * np.sin(w2 * tv) + A3 * np.cos(w3 * tv - np.pi)

# Graficar la señal continua original
plt.plot(tv, X, 'r', label='Señal continua $x(t)$')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Señal continua $x(t)$')
plt.legend()
plt.grid(True)
plt.show()


# se grafica la funcion orginal para tener un contraste con respuestas futuras

In [ ]:
def my_ceropen(x,ymin = -5, ymax = 5): #en general se pueden definir valores por defecto
  '''
  Codigo base para simular proceso de cero y pendiente
  Se ingresa arreglo de numpy y los valores min  y max despues de cero y pendiente
  '''
  xmax = max(x) # x.max()
  xmin = min(x) # x.min()
  m = (ymax - ymin)/(xmax - xmin)
  c = ymin - m*xmin
  yv = m*x + c
  return yv

In [ ]:
ymin=-3.3
ymax=3.3
xcp= my_ceropen(X,ymin=ymin,ymax=ymax)
plt.plot(tv,xcp,c='r',label='$cero_pendiente$')
plt.legend()
plt.show()

In [ ]:
def my_cuantizador(yn, vq) : #yn punto a #cuantizar, vq vector de estados

  Ne = vq.shape[0] #tamaño vector de estados
  dn = cdist(yn.reshape(-1,1),vq.reshape(-1,1))#distancia yn a vector estados, reshape(-1,1) asegura vectores columna para poder utilizar cdist
  ind = np.argmin(dn) #posición distancia min
  return vq[ind]

In [ ]:
#función cuantización punto a punto
import numpy as np
import matplotlib.pyplot as plt


#____________________________________________________________________________
for i in range(xcp.shape[0]):
  if i==0: #primera muestra
    yq = my_cuantizador(xcp[i],vq)
  else:
    yq = np.append(yq,my_cuantizador(xcp[i],vq)) #apilar datos cuantizando si i>0
#_____________________________________________________________________________
#graficar señales

plt.plot(tv,xcp,c='r', label='ycs')#cs continua

plt.stem(tv,xcp, label='ycs_dis')#cs discreta

#estados de cuantización

for v in vq:
  plt.plot(tv,v*np.ones(len(tv)))

# graficar señal Digital (discreta + cuantizada)

plt.plot(tv,yq,c='y',marker='o',label='ydig', linewidth=2)

plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.show()

Se observa que se cumple nyquist y se observa que la señal esta completa

# Pregunta 2 (valor 2.5 puntos)

Se dispone de un sistema modelado como una "caja negra" (ver celdas de código). Su tarea es analizar y comprobar mediante simulaciones si el sistema cumple con las propiedades de linealidad e invariancia en el tiempo. En caso de que el sistema sea lineal e invariante con el tiempo, determine su respuesta al impulso y utilice esta respuesta para calcular la salida del sistema ante la siguiente señal:

$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
# cargar sistema
FILEID = "1J9rhh0wWHZSBd8XmWGt1ZpCsMDuoUFmm"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O P1_model.zip && rm -rf /tmp/cookies.txt
!unzip -o P1_model.zip
!dir

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from P1_model import system_
from scipy.signal import firwin, freqz, lfilter, filtfilt



#sistema pregunta 2
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs #frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo
signal_u = np.heaviside(t,1) # función heaviside
y_u = my_system.predict(signal_u)
# Visualización de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, signal_u, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_u, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from P1_model import system_
from scipy.signal import firwin, freqz, lfilter, filtfilt

# Parámetros
A = 2  # Amplitud del escalón
B = 5  # Amplitud de la sinusoide
f =120  # Frecuencia de la sinusoide

# Sistema
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs  # Frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo

# Señales de entrada
signal_u1 = np.heaviside(t, 0)  # Escalón unitario
signal_sin = np.sin(2 * np.pi * f * t)  # Sinusoide
signal_esp = A*signal_u1 + B*signal_sin  # Combined signal

# Respuestas del sistema
y_u1 = my_system.predict(signal_u1)
y_sin = my_system.predict(signal_sin)
Y_e = my_system.predict(signal_esp)  # Output for combined signal
signal_p = A*y_u1 + B*y_sin #suma de las salidas individuales

# Visualización


# Graficar señales y salidas
plt.figure(figsize=(12, 8))

# Gráfica de las señales de entrada
plt.subplot(3, 1, 1)
plt.plot(t, signal_u1, label="Escalón Unitario (Signal_u1)")
plt.plot(t, signal_sin, label="Sinusoide (Signal_sin)")
plt.title("Señales de Entrada")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid()

# Gráfica de las salidas individuales
plt.subplot(3, 1, 2)
plt.plot(t, y_u1, label="Salida por Escalón (Y_u1)")
plt.plot(t, y_sin, label="Salida por Sinusoide (Y_sin)")
plt.title("Salidas Individuales")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid()

# Comprobacion
plt.subplot(3, 1, 3)
plt.plot(t, signal_p, label="Salida Sistema para suma del todo", linestyle="-")
plt.plot(t, Y_e, label="Salida Esperada por Linealidad ", linestyle="--")
plt.title("Comparación: Y_L vs Y_e")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


se puede observar la linealidad debido a que las señales se superponene indicando que son la misma evaluada en el todo y evaluda POR PARTES

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from P1_model import system_
from scipy.signal import firwin, freqz, lfilter, filtfilt

# Parámetros
A = 2  # Amplitud del escalón
B = 5  # Amplitud de la sinusoide
f =120  # Frecuencia de la sinusoide

# Sistema
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs  # Frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo


# Señales de entrada
signal_u1 = np.heaviside(t+2, 0)  # Escalón unitario
signal_sin = np.sin(2 * np.pi * f * t+2)  # Sinusoide
signal_esp = A*signal_u1 + B*signal_sin  # Combined signal

# Respuestas del sistema
y_u1 = my_system.predict(signal_u1)
y_sin = my_system.predict(signal_sin)
Y_e = my_system.predict(signal_esp)  # Output for combined signal
signal_p =( A*y_u1 + B*y_sin) #suma de las salidas individuales

# Visualización


# Graficar señales y salidas
plt.figure(figsize=(12, 8))

# Gráfica de las señales de entrada
plt.subplot(3, 1, 1)
plt.plot(t, signal_u1, label="Escalón Unitario (Signal_u1)")
plt.plot(t, signal_sin, label="Sinusoide (Signal_sin)")
plt.title("Señales de Entrada")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid()

# Gráfica de las salidas individuales
plt.subplot(3, 1, 2)
plt.plot(t, y_u1, label="Salida por Escalón (Y_u1)")
plt.plot(t, y_sin, label="Salida por Sinusoide (Y_sin)")
plt.title("Salidas Individuales")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid()

# Comprobacion
plt.subplot(3, 1, 3)
plt.plot(t, signal_p, label="Salida Sistema para suma del todo", linestyle="-")
plt.plot(t, Y_e, label="Salida Esperada por Linealidad ", linestyle="--")
plt.title("Comparación: Y_L vs Y_e")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

en este momento se cambia el tiempo en t+2 y se obtiene la misma señal comnprobando invarianza en el tiempo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from P1_model import system_

# Crear el sistema
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs  # Frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo
signal_u = np.heaviside(t, 1)  # Función Heaviside (escalón unitario)
y_u = my_system.predict(signal_u)  # Respuesta al escalón
y_d = np.diff(y_u)  # Aproximación de la respuesta al impulso

ts=1/fs
n=np.arange(len(y_d))
x_n = np.sin(100 * np.pi * n/fs) + np.sin(600 * np.pi * n/fs)

# Realizar la convolución entre y_d (respuesta al impulso) y x_n (entrada)
conv_result = np.convolve(y_d, x_n, mode='full')  # 'full' devuelve la convolución completa

# Graficar las señales originales y la convolución
plt.figure(figsize=(12, 8))

# Señal de entrada x_n
plt.subplot(3, 1, 1)
plt.plot(n/fs, x_n[:len(t)], label="Entrada x_n (ampliada)", color='b')
plt.title("Señal de Entrada: x_n (Ampliada)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid()

# Respuesta al impulso y_d
plt.subplot(3, 1, 2)
plt.plot(t[1:], y_d, label="Respuesta al impulso y_d")  # t[1:] para ajustar el tamaño
plt.title("Respuesta al Impulso: y_d")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid()

# Resultado de la convolución
plt.subplot(3, 1, 3)
plt.plot(np.arange(-0.01, 0.02 + (len(conv_result) - len(t)) / fs, 1/fs), conv_result, label="Convolución de y_d y x_n")
plt.title("Resultado de la Convolución")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


se raliza la derivada de la señal escalon para obtener la señal impusol y se realiza la convolucion con la señal dada por el ejercicio para la cual al estar en tiempo discreto se debe realizar el despeje de n en funcion del tiempo obteniendo que n/fs obtenido de la formula t=n*ts  debido a que ts = 1/fs tenemos que t=n/fs y de esta manera se puede simular la señal deseada
